In [17]:
import cv2
import numpy as np
import random

class MotionCaptureData:
    def __init__(self):
        self.marker_positions = []

    def add_positions(self, positions):
        self.marker_positions.append(positions)

    def get_marker_positions(self):
        return self.marker_positions

def replace_blue_background(frame, background, lower_blue, upper_blue):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower_blue, upper_blue)
    mask = cv2.dilate(mask, np.ones((5, 5), np.uint8), iterations=1)
    mask_inv = cv2.bitwise_not(mask)
    
    foreground = cv2.bitwise_and(frame, frame, mask=mask_inv)
    background_resized = cv2.resize(background, (frame.shape[1], frame.shape[0]))
    
    background_area = cv2.bitwise_and(background_resized, background_resized, mask=mask)
    result = cv2.add(foreground, background_area)
    return result

def segment_red_markers(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower_red1, upper_red1 = np.array([0, 120, 70]), np.array([10, 255, 255])
    lower_red2, upper_red2 = np.array([170, 120, 70]), np.array([180, 255, 255])
    mask = cv2.inRange(hsv, lower_red1, upper_red1) + cv2.inRange(hsv, lower_red2, upper_red2)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((5, 5), np.uint8))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((5, 5), np.uint8))
    return mask

def detect_markers(mask):
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    positions = []
    for contour in contours:
        if cv2.contourArea(contour) > 50:  
            M = cv2.moments(contour)
            if M["m00"] > 0:
                cx = int(M["m10"] / M["m00"])
                cy = int(M["m01"] / M["m00"])
                positions.append((cx, cy))
    return positions

backgrounds = [cv2.imread("background1.jpg"), cv2.imread("background2.jpg"), cv2.imread("background3.jpg")]

In [18]:
class BouncingBall:
    def __init__(self, position, radius, color, speed):
        self.position = np.array(position)
        self.radius = radius
        self.color = color
        self.speed = np.array(speed)

    def move(self, frame_size):
        self.position += self.speed
        if self.position[0] <= self.radius or self.position[0] >= frame_size[1] - self.radius:
            self.speed[0] = -self.speed[0]
        if self.position[1] <= self.radius or self.position[1] >= frame_size[0] - self.radius:
            self.speed[1] = -self.speed[1]

    def draw(self, frame):
        cv2.circle(frame, tuple(self.position.astype(int)), self.radius, self.color, -1)

class FloatingBalloon:
    def __init__(self, position, size, color, speed):
        self.position = np.array(position)
        self.size = size  
        self.color = color
        self.speed = np.array(speed)

    def move(self, frame_size):
        self.speed += np.random.randint(-2, 3, size=2)  
        self.position += self.speed
        
        self.position[0] = np.clip(self.position[0], 0, frame_size[1] - self.size[0])
        self.position[1] = np.clip(self.position[1], 0, frame_size[0] - self.size[1])

    def draw(self, frame):
        center = (int(self.position[0] + self.size[0] // 2), int(self.position[1] + self.size[1] // 2))
        axes = (self.size[0] // 2, self.size[1] // 2)  # Half of width and height for ellipse axes
        cv2.ellipse(frame, center, axes, angle=0, startAngle=0, endAngle=360, color=self.color, thickness=-1)




In [19]:
class CharacterPart:
    def __init__(self, position, size, color=(0, 0, 255)):  
        self.position = np.array(position, dtype=np.int32)
        self.size = size
        self.color = color
    
    def draw(self, frame):
        cv2.circle(frame, tuple(self.position), self.size, self.color, -1)
    
    def move_to(self, new_position):
        self.position = np.array(new_position, dtype=np.int32)

class Character:
    def __init__(self):
        self.body = CharacterPart(position=[300, 300], size=20, color=(0, 0, 255))   
        self.left_arm = CharacterPart(position=[280, 320], size=10, color=(0, 255, 0))  
        self.right_arm = CharacterPart(position=[320, 320], size=10, color=(0, 255, 0)) 
        self.left_leg = CharacterPart(position=[290, 340], size=10, color=(255, 0, 0))  
        self.right_leg = CharacterPart(position=[310, 340], size=10, color=(255, 0, 0)) 
    
    def draw(self, frame):
        self.body.draw(frame)
        self.left_arm.draw(frame)
        self.right_arm.draw(frame)
        self.left_leg.draw(frame)
        self.right_leg.draw(frame)
    
    def update_positions(self, positions):
        if len(positions) >= 5:
            self.body.move_to(positions[0])
            self.left_arm.move_to(positions[1])
            self.right_arm.move_to(positions[2])
            self.left_leg.move_to(positions[3])
            self.right_leg.move_to(positions[4])

In [20]:
def check_collision(obj1, obj2):
    distance = np.linalg.norm(obj1.position - obj2.position)
    return distance <= (obj1.radius + obj2.radius)

def lightning_strike_event(frame, ball, balloon):
    lightning_flash = np.full(frame.shape, (255, 255, 255), dtype=np.uint8)
    cv2.addWeighted(lightning_flash, 0.5, frame, 0.5, 0, frame)
    
    ball.speed *= 2
    balloon.speed += np.array([3, -2])
    ball.speed //= 2 
    balloon.speed -= np.array([3, -2])


def gravity_shift_event(frame, ball, balloon):
    background_shift = np.full(frame.shape, (0, 0, 128), dtype=np.uint8)
    cv2.addWeighted(background_shift, 0.5, frame, 0.5, 0, frame)
    ball.speed = np.array([random.randint(-5, 5), random.randint(-5, 5)])
    balloon.speed = np.array([random.randint(-2, 2), random.randint(-2, 2)])

def process_video_with_intelligent_objects(video_path, output_path):
    cap = cv2.VideoCapture(video_path)
    fps = 30
    total_frames = 1800  
    lower_blue = np.array([100, 150, 0])
    upper_blue = np.array([140, 255, 255])
    
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

    backgrounds = [cv2.imread("background1.jpg"), cv2.imread("background2.jpg"), cv2.imread("background3.jpg")]

    ball = BouncingBall(position=[100, 100], radius=20, color=(0, 0, 255), speed=[5, 5])
    balloon = FloatingBalloon(position=[400, 300], size=(50, 100), color=(255, 0, 0), speed=[2, -1])

    sid_text = "SID530509598_Asgmt2Opt1"
    text_position = (10, 30) 
    text_color = (0, 0, 255)  
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.8
    thickness = 2

    frame_count = 0
    while cap.isOpened() and frame_count < total_frames:
        ret, frame = cap.read()
        if not ret:
            cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  
            continue

        marker_mask = segment_red_markers(frame)
        marker_positions = detect_markers(marker_mask)
        current_background = backgrounds[frame_count // 100 % len(backgrounds)]

        frame = replace_blue_background(frame, current_background, lower_blue, upper_blue)

        for position in marker_positions:
            cv2.circle(frame, position, 5, (0, 0, 255), -1)  

        cv2.putText(frame, sid_text, text_position, font, font_scale, text_color, thickness)

        if frame_count % 300 == 0:
            lightning_strike_event(frame, ball, balloon)

        if frame_count % 600 == 0:
            gravity_shift_event(frame, ball, balloon)

        frame_size = frame.shape[:2]
        ball.move(frame_size)
        balloon.move(frame_size)
        
        ball.draw(frame)
        balloon.draw(frame)

        out.write(frame)

        cv2.imshow('Intelligent Objects with Character Replacement', frame)

        if cv2.waitKey(int(1000 / fps)) & 0xFF == ord('q'):
            break

        frame_count += 1

    cap.release()
    out.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    video_path = "Opt1-MarionetteMovements.mov"
    output_path = "output_video.mp4"
    process_video_with_intelligent_objects(video_path, output_path)